<a href="https://colab.research.google.com/github/dienerlab/2024_micom_intro/blob/main/micom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧫🦠 Modeling microbiota-wide metabolism with MICOM

This notebook will accompany the second MICOM intro session. The presentation slides can be [found here](https://dienerlab.github.io/2024_micom_intro). 

You can save your own local copy of this notebook by using `File > Save a copy in Drive`. You may be prompted to certify the notebook is safe. We promise that it is 🤞

**Disclaimer:**
The linear and quadratic programming problems MICOM has to solve are very large and very complicated. There are some very good commercial solvers that are very expensive (even though they are often free for academic use). To make this tutorial as accessible as possible we will use an Open Source hybrid solver developed for MICOM. This solvr is accurate but may be slower than commercial solvers. Installing cost-free academic versions of commercial solvers like [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [Gurobi](https://www.gurobi.com/) will speed up some of the computations.



# 📝 Setup

MICOM installation is is usually pretty straight-forward and can be as easy as typing `pip install micom` into your Terminal. This will also work in a conda environment for example.

First let's start by downloading the materials and switching to the folder.

In [ ]:
!git clone https://github.com/dienerlab/2024_micom_intro materials
%cd materials

## Basic Installation

Installing MICOM is straight-forward in Python. The solver itself will be installed automatically along with it. This will also work in a conda environment.

In [ ]:
!pip install -q micom

print("Done! 🎉 ")

Okay, all done. So let's get started building some models 🦺🛠d😁.

# 💻 MICOM

We will use the Python interface to MICOM since it plays nicely with Colaboratory.

Here is an overview of all the steps and functions across both interfaces:
![micom overview](https://github.com/micom-dev/q2-micom/raw/706f583a060b91c12c0cec7acea2354fdd0dd320/docs/assets/overview.png)

The process of building a metabolic model in MICOM begins with constructing a combined abundance/taxonomy table, referred to hereafter as a taxonomy table. Let's load our taxonomy table to see what it looks like: 



In [ ]:
import pandas as pd

table = pd.read_csv("tax_table.csv")
table


In this taxonomy table, we two samples. The first sample is from an healthy individual in the iHMP cohort. The sample was processed with Kraken2 and Bracken and the abundance denote reads assigned to that species. The second sample is exactly the same abundances with an additional *Bifidobacterium animalis* entry. This is one of the strengths of MICOM. It simulates *single samples* so you can always add in samples with arbitrary modification such as drop-ins and drop-outs for example. The `id` columns is an alternative name for the taxon that has to fulfill certain formatiing requirements (no spaces). It will be optional in the next MICOM version.

Your table can also contain additional, arbitrary columns. Those will be passed through the MICOM analyses.

The specific format you see here is called [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). It might be useful to get familiar with that concept at some point, but for now it is important to know that all primary outputs of MICOM workflows will also be in that format (and the rest will soon be converted to it as well). 

## Building Microbial Community Metabolic Models (MCMMs)

If we only have abundances we still need the individuals reconstructions for the species. This is provided in a *model database*. One important thing to to remember is that the taxonomy format for the database and your table *has to be the same*. So if you have Kraken2 data you need a database in RefSeq format (using the NCBI taxonomy). If your table would be GTDB taxa you would need to use a database for that. The rest of the name tells you where the models came from. We will use AGORA2, a database of manually curated metabolic models for several thousand strains. There are many databases provided in the [MICOM model databases](https://zenodo.org/records/7739096). Let's download the species one for AGORA2.

In [ ]:
!wget "https://zenodo.org/records/7739096/files/agora201_refseq216_species_1.qza?download=1" -O agora201_refseq216_species_1.qza

Now we're all set to start building models! We only have two samples so we will build them in parallel. 

In [ ]:
from micom.workflows import build

manifest = build(table, "agora201_refseq216_species_1.qza", "models", threads=2)

MICOM will warn you if less than 50% of the abundances can be matched to the database for one of the samples. This can happen with some data, and may indicate the models may not be completely representative of the samples. Typically a fraction of 80% or more is considered great. We'll continue, but remember to keep an eye out for this in future projects!

In lower-biomass 16S amplicon sequencing samples from stool, many reads can match to food components or to host mitochondria and these hits probably do not contribute much to bacterial community metabolism. These hits will be excluded from MICOM. 

Let's take a look what we got back from the `build` process.

In [ ]:
manifest

This will tell you how many taxa were found in the database and what fraction of the total abundance was represented by the database. Here all species were in the database, stellar!

So we now have our community models and can leverage MICOM fully by simulating community growth - let's discuss what we want to look at. 

### Dietary Context

MICOM always requires environmental context, meaning data concerning availability of nutrients in the environment. This is considered a feature not a bug. In the end nothing in ecology makes sense except in the light of the environment.

For gut microbiome samples, this is often coming from the diet, and there are many environmental media formulations for MICOM available. This time we need to match those to the *source* of the model database (the taxonomy naming does not matter here). So we will need a medium that is built for AGORA2. Luckily there already is one for an average european diet right here, phew. The model is save in a Qiime2 format but we can still load it here directly (even without a Qiime2 installation).

In [ ]:
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("average_european_agora2.qza")
medium

### Growing the models
Great, now we have our media & our models, it's time to get growing. We'll grow of the samples in the same medium. This is way more controlled than any real study we could run. All differences we see are purely because of the added probiotic. This will take some time, so we'll use that time as an opportunity to discuss more in depth what these processes do, and what to look for in the results. First, let's run the `grow()` command using the matched medium. This will take the models we've built, and find an optimal solution to the fluxes based upon the medium that's been applied.

In [ ]:
from micom.workflows import grow, save_results

results = grow(manifest, "models", medium, tradeoff=0.5, threads=2)

# We'll save the results to a file
save_results(results, "results.zip")

Again, if that takes too long or was aborted, we can read it in from a cached result.

In [ ]:
from micom.workflows import load_results

try:               # Will only run if the previous step failed
  results
except NameError:
  matched_growth = load_results("results.zip")

What kind of results did we get? Well, `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases

### 📈 Growth Rates

The growth rates are pretty straightforward. What can you say about the probiotic species we added?

In [ ]:
results.growth_rates

### ↔️ Exchange Fluxes 

More interesting are the exchange fluxes. These reactions represent the import and export of metabolites from the system Let's look at those now:

In [ ]:
results.exchanges

So we see how much of each metabolite is either consumed or produced by each taxon in each sample. `tolerance` denotes the accuracy of the solver and tells you the smallest absolute flux that is likely different form zero (i.e., substantial flux). *All of the fluxes are normalized to 1g dry weight of bacteria*. So, you can directly compare fluxes between taxa, even if they are present at very different abundances. 

If you're curious what the abbreviation for each of these metabolites represents, that can be found in the annotations dataframe. For instance, let's find out what `"but[e]"` represents. 

In [ ]:
anns = results.annotations
anns[anns.metabolite == "but[e]"]

Butyrate! Interesting, [that's an important metabolite](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8608412/)! All of these annotations and more information at are also available at https://vmh.life, maintained by Dr. Ines Thiele's lab. 

We can also look at production rates for metabolites. Let's do that and look at valine.

In [ ]:
from micom.measures import production_rates

prod = production_rates(results)
prod[prod.name == "L-valine"]

Finally we could go back to the exchange fluxes to see who produces the valine.

In [ ]:
ex = results.exchanges
ex[ex.metabolite == "val_L[e]"].sort_values(by="flux", ascending=False)

Let's stop here for today and feel free to check out the other online courses that will go into much more depth.